## Carte du taux d'incidence

In [ ]:
import ipyleaflet as ipyl
import ipywidgets as ipyw

In [ ]:
from plot_covid19_fr.cartes import data_geojson_dep, get_incidence, get_positivity, color_dep, cmap

In [ ]:
m = ipyl.Map(center=(47.5, 2), zoom=6,
             layout=ipyw.Layout(width='100%', height='670px'))

geo_json = ipyl.GeoJSON(
    data=data_geojson_dep,
    style={
        'opacity': 1, 'dashArray': '0', 'fillOpacity': 0.6, 'weight': 1
    },
    hover_style={
        'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5
    },
    style_callback=color_dep
)

label = ipyw.Label(layout=ipyw.Layout(width='100%'))
label.value = "??"

def click_handler(event=None, feature=None, id=None, properties=None):
    dep = feature["properties"]["code"]
    dep_name = feature["properties"]["nom"]
    incidence = get_incidence(dep)
    positivity = get_positivity(dep)
    label.value = (f"{dep_name} ({dep}) : taux d'incidence = {incidence:.0f}, "
    f"taux de positivité = {positivity:.2f} %")
    
    
geo_json.on_click(click_handler)

m.add_layer(geo_json)

out = ipyw.widgets.Output(layout={'border': '1px solid black'})
with out:
    display(cmap)
    
widget_control = ipyl.WidgetControl(widget=out, position='topright')
m.add_control(widget_control)

out2 = ipyw.widgets.Output(layout={'border': '1px solid black'})
with out2:
    display(label)

widget_control2 = ipyl.WidgetControl(widget=out2, position='topleft')
m.add_control(widget_control2)
m